# MAPRR Textual Analytics

## Intro

### Import

In [1]:
import os 
import time
import logging
import pandas as pd 
import re
import threading
import requests
from bs4 import BeautifulSoup
from natasha import (
    Segmenter, 
    MorphVocab, 
    NewsEmbedding, 
    NewsMorphTagger, 
    NewsSyntaxParser, 
    NewsNERTagger, 
    PER, 
    NamesExtractor, 
    Doc)
from razdel import tokenize

In [2]:
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

## Pre-processing Function

In [3]:
domain = 'https://mpgrr.herokuapp.com/'

In [4]:
tables = {'agents/': 304,
          'works/': 603, 
          'place_based_concepts/': 315, 
          'locations/': 366, 
          'multivalent_markers/': 433}

In [5]:
class maprr: 
    
    def __init__(self): 
        self.Wsoup = {} 
        self.Asoup = {}
        self.Ws = {}
        self.As = {}
    
    def get_htmlA(self): 
        # list(tables.values())[0]
        for i in range(1, list(tables.values())[0]+1):
            url = domain+list(tables.keys())[0]+str(i)
            with requests.get(url) as r: 
                if r.status_code == 200: 
                    s = BeautifulSoup(r.content, 'html.parser')
                    self.Asoup.update({i:s})
                    #print(f"Appending A{i}")
                    logging.info(f"Appending A{i}")
                    time.sleep(.1)
                else: 
                    print(f"A{i} status code: {r.status_code}")
                    time.sleep(.1)
                    pass

    def get_htmlW(self): 
        # list(tables.values())[1]
        for i in range(1, list(tables.values())[1]+1):
            url = domain+list(tables.keys())[1]+str(i)
            with requests.get(url) as r: 
                if r.status_code == 200: 
                    s = BeautifulSoup(r.content, 'html.parser')
                    self.Wsoup.update({i:s})
                    #print(f"Appending W{i}")
                    logging.info(f"Appending W{i}")
                    time.sleep(.1)
                else: 
                    print(f"W{i} status code: {r.status_code}")
                    time.sleep(.1)
                    pass

    def parseWs(self, html): 
        content = html.find('div', {'class':'col-md-9 fixed-height'})
        try: 
            author = content.div.h3.text
        except: 
            author = ""
        try: 
            title = content.div.h4.text
        except: 
            title = ""
        text = content.find_all('p',{'class':'stanza'})
        Wtext = [x.text.replace('\n','').strip() for x in text]
        metaKeys = [x.text[:-1] for x in html.find('div', {'class':'card-body'}).find_all('h4')]
        metaVals = [x.text for x in html.find('div', {'class':'card-body'}).find_all('p')]
        metaDict = dict(zip(metaKeys, metaVals))
        subDict = {'title': title, 'text': Wtext}
        subDict.update(metaDict)
        #self.Ws.update(subdict)
        return subDict

    def parseAs(self, html): 
        name = html.find('div', {'class': 'card scrollable'}).h2.text
        bdate, ddate = html.find('div', {'class': 'card scrollable'}).span.text.split(' - ')
        subDict = {'name': name, 'birth': bdate, 'death': ddate}
        try: 
            typeKeys = [x.h4.text for x in html.find_all('div', {'class': 'col-md-4'})]
            typeVals = [x.p or x.div.span.text for x in html.find_all('div', {'class': 'col-md-4'})]
            typeVals[:2] = [x.text for x in typeVals[:2]]
            typeDict = dict(zip(typeKeys, typeVals))
            subDict.update(typeDict)
        except: 
            pass
        #self.As.update(subdict)
        return subDict

    def run(self): 
        self.get_htmlA() 
        self.get_htmlW() 

        self.As = {k: self.parseAs(v) for k, v in self.Asoup.items()}
        self.Ws = {k: self.parseWs(v) for k, v in self.Wsoup.items()}

        AsDf = pd.DataFrame.from_dict(self.As, orient='index')
        WsDf = pd.DataFrame.from_dict(self.Ws, orient='index')        

        WsDf.to_csv('WsDf.csv')
        AsDf.to_csv('AsDf.csv')

In [6]:
%%time
if __name__ == '__main__': 
    maprr().run()

INFO:Appending A1
INFO:Appending A2
INFO:Appending A3
INFO:Appending A4
INFO:Appending A5
INFO:Appending A6
INFO:Appending A7
INFO:Appending A8
INFO:Appending A9
INFO:Appending A10
INFO:Appending A11
INFO:Appending A12
INFO:Appending A13
INFO:Appending A14
INFO:Appending A15
INFO:Appending A16
INFO:Appending A17
INFO:Appending A18
INFO:Appending A19
INFO:Appending A20
INFO:Appending A21
INFO:Appending A22
INFO:Appending A23
INFO:Appending A24
INFO:Appending A25
INFO:Appending A26
INFO:Appending A27
INFO:Appending A28
INFO:Appending A29
INFO:Appending A30
INFO:Appending A31
INFO:Appending A32
INFO:Appending A33
INFO:Appending A34
INFO:Appending A35
INFO:Appending A36
INFO:Appending A37
INFO:Appending A38
INFO:Appending A39
INFO:Appending A40
INFO:Appending A41
INFO:Appending A42
INFO:Appending A43
INFO:Appending A44


A45 status code: 500


INFO:Appending A46
INFO:Appending A47
INFO:Appending A48
INFO:Appending A49
INFO:Appending A50
INFO:Appending A51
INFO:Appending A52
INFO:Appending A53
INFO:Appending A54
INFO:Appending A55
INFO:Appending A56
INFO:Appending A57
INFO:Appending A58
INFO:Appending A59
INFO:Appending A60
INFO:Appending A61
INFO:Appending A62
INFO:Appending A63
INFO:Appending A64
INFO:Appending A65
INFO:Appending A66
INFO:Appending A67
INFO:Appending A68
INFO:Appending A69
INFO:Appending A70
INFO:Appending A71
INFO:Appending A72
INFO:Appending A73


A74 status code: 404
A75 status code: 404
A76 status code: 404
A77 status code: 404


INFO:Appending A78
INFO:Appending A79
INFO:Appending A80
INFO:Appending A81
INFO:Appending A82
INFO:Appending A83
INFO:Appending A84
INFO:Appending A85
INFO:Appending A86
INFO:Appending A87
INFO:Appending A88
INFO:Appending A89
INFO:Appending A90
INFO:Appending A91
INFO:Appending A92
INFO:Appending A93
INFO:Appending A94
INFO:Appending A95
INFO:Appending A96
INFO:Appending A97
INFO:Appending A98
INFO:Appending A99
INFO:Appending A100
INFO:Appending A101
INFO:Appending A102
INFO:Appending A103
INFO:Appending A104
INFO:Appending A105
INFO:Appending A106
INFO:Appending A107
INFO:Appending A108
INFO:Appending A109
INFO:Appending A110
INFO:Appending A111
INFO:Appending A112
INFO:Appending A113
INFO:Appending A114
INFO:Appending A115
INFO:Appending A116
INFO:Appending A117
INFO:Appending A118
INFO:Appending A119
INFO:Appending A120
INFO:Appending A121
INFO:Appending A122
INFO:Appending A123
INFO:Appending A124
INFO:Appending A125
INFO:Appending A126
INFO:Appending A127
INFO:Appending A128
IN

...

W581 status code: 500


INFO:Appending W582
INFO:Appending W583
INFO:Appending W584
INFO:Appending W585
INFO:Appending W586
INFO:Appending W587
INFO:Appending W588
INFO:Appending W589
INFO:Appending W590
INFO:Appending W591
INFO:Appending W592
INFO:Appending W593
INFO:Appending W594
INFO:Appending W595
INFO:Appending W596
INFO:Appending W597


W598 status code: 500


INFO:Appending W599
INFO:Appending W600
INFO:Appending W601
INFO:Appending W602
INFO:Appending W603


CPU times: user 1min 27s, sys: 1.69 s, total: 1min 29s
Wall time: 13min 54s


In [7]:
WsDf = pd.read_csv('WsDf.csv').drop(columns=['Unnamed: 0'])
WsDf.index.name = 'W#'
WsDf

,title,text,Title,First Line,Author,Composition Date,Composition Location,Source of First Publication,First Publication Publisher,First Publication Year,First Publication Location
W#,,,,,,,,,,,
0,Untitled,"['Сразу стало тихо в доме, Обле...",no title,Srazu stalo tikho v dome…,Anna Akhmatova,July 1917,Slepnevo,Podorozhnik,Petropolis,1921,NaN
1,Untitled,['Ты — отступник: за остров зелёный ...,no title,Ty - otstupnik: za ostrov zelenyi…,Anna Akhmatova,summer 1917,Slepnevo,Podorozhnik,Petropolis,1921,NaN
2,Untitled,['Просыпаться на рассвете Оттог...,no title,Prosypat'sia na rassvete…,Anna Akhmatova,July 1917,Slepnevo,Podorozhnik,Petropolis,1921,NaN
3,Untitled,"['И в тайную дружбу с высоким, ...",no title,I v tainuiu druzhbu c vysokim…,Anna Akhmatova,1917,Petrograd,Podorozhnik,Petropolis,1921,NaN
4,Untitled,"['Словно ангел, возмутивший воду, ...",no title,"Slovno angel, vozmutivshii vodu…",Anna Akhmatova,February 1916,Tsarskoe selo,Podorozhnik,Petropolis,1921,NaN
...,...,...,...,...,...,...,...,...,...,...,...
581,Untitled,"['Любовь распяли на кресте, Но в...","""Liubov' raspiali na kreste""",Liubov' raspiali na kreste,Georgii Andreevich Viatkin,NaN,Omsk,Ranenaia Rossiia: Stikhi; Vernost': rasskaz; E...,Tipografiia Vremennogo Tsentral’nogo Voenno-P...,1919,Ekaterinburg
582,На словах...,[],Na slovakh... (Nesvoevremennye mysli),Na slovakh--vse soglasny...,Maksim Gor'kii,"June 29, 1917",Petrograd,Novaia zhizn',A. N. Tikhonov,"June 29, 1917",Petrograd
583,Последняя просьба,"['Сестра!.. Сестрица, на минутку подойдите ...",Posledniaia pros'ba,"Sestra! Sestritsa, na minutku podoidite…",M Kolchin,date unknown,NaN,Pesni voiny: posviashchaetsia doblestnym sibir...,Tipografiia I. M. Poznera,1915,NaN


In [8]:
AsDf = pd.read_csv('AsDf.csv').drop(columns=['Unnamed: 0'])
AsDf.index.name = 'A#'
AsDf

,name,birth,death,Type of Agent,Sex,Occupations,Family Social Strata,Literary Affiliations,Political Affiliations,Type of Corporate Body,Affiliation
A#,,,,,,,,,,,
0,Anna Akhmatova,"June 23, 1889","March 5, 1966",person,female,poet,nobility,Acmeism,independent,NaN,NaN
1,Vasilii Dmitrievich Aleksandrovskii,"January 15, 1897","November 16, 1934",person,male,soldier,peasant,Kuznitsa,Bolshevik member,NaN,NaN
2,Ivan Nikolaevich Antonov,1878,1936?,person,male,editor,unknown,unknown,independent,NaN,NaN
3,Mikhail Dmitrievich Artamonov,"February 22, 1888","November 22, 1958",person,male,journalist,peasant,Vologda poets,unknown,NaN,NaN
4,Nikolai Aseev,"July 10, 1889","July 16, 1963",person,male,soldier,nobility,Left Front of Art: LEF,Bolshevik member,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
285,Moisei Solomonovich Uritskii,"January 14, 1873","August 30, 1918",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286,Maximilien Marie Isidore de Robespierre,"May 6, 1758","June 28, 1794",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287,Iurii Mikhailovich Steklov,"August 27, 1873","July 15, 1941",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Dataframe Split

### libDf

In [ ]:
libDf = pd.DataFrame.from_dict(XMLdict, orient='index').rename_axis('work_num').sort_index(inplace=False)
libDf.index = libDf.index.str.lstrip('0')
libDf

### authorsDf

In [ ]:
authorsDf = libDf.reset_index().groupby('author').size().to_frame().rename(columns={0:'num_works'})
authorsDf[['num_lps','num_words']] = libDf.reset_index().groupby('author').sum()
authorsDf.sort_values(by=['num_words','num_works'], ascending=False)
authorsDf

### worksDf

In [ ]:
worksDf = libDf[['title','year','author','genre','num_lps','num_words']]
worksDf

### tokenDf

In [ ]:
lpDf = libDf[['text']]
lpDf = lpDf.text.apply(lambda x: pd.Series([y for y in x])).stack().to_frame().rename(columns={0:'lp_str'})
lpDf.index.names = OHCO[:2]
lpDf
tokenDf = lpDf.lp_str.apply(lambda x: tokenize(x)).to_frame()#.rename(columns={0:'token'})
#tokenDf = lpDf.lp_str.apply(lambda x: y.text for y in tokenize(x)[1])
tokenDf

In [ ]:
for i in tokenize(lpDf.lp_str): 
    print(i)